### Модуль предобработки данных. 

In [1]:
#pip install scikit-learn==1.1
#%pip install --upgrade scikit-learn==1.4.0
#

In [1]:
import pandas as pd
import os
import pickle
import numpy as np

from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from numpy import genfromtxt

##### Имитируем подачу данных с датчиков

In [2]:
General_path = "C:/Users/Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/"
raw = "raw/"
processed = "processed/"
final = "final/"
tests = "tests/"

# train_FD001 = pd.read_csv(General_path + "train_FD001.csv")
# train_FD002 = pd.read_csv(General_path + "train_FD002.csv")
# train_FD003 = pd.read_csv(General_path + "train_FD003.csv")
# train_FD004 = pd.read_csv(General_path + "train_FD004.csv")

# test_FD001 = pd.read_csv(General_path + "test_FD001.csv")
# test_FD002 = pd.read_csv(General_path + "test_FD002.csv")
# test_FD003 = pd.read_csv(General_path + "test_FD003.csv")
# test_FD004 = pd.read_csv(General_path + "test_FD004.csv")

# This dataset using for show preprocess handling incoming data
# BAD_test_FD001 = pd.read_csv(General_path + "BAD_test_FD001.csv")

# Small_BAD_test_FD001 = pd.read_csv(General_path + raw + "Small_BAD_test_FD001.csv")
np_Small_BAD_test_FD001 = genfromtxt(General_path + raw + "Small_BAD_test_FD001.csv", delimiter=',')
np_Small_GOOD_test_FD001 = genfromtxt(General_path + raw + "Small_GOOD_test_FD001.csv", delimiter=',')
# Small_GOOD_test_FD001 = pd.read_csv(General_path + "Small_GOOD_test_FD001.csv")

##### Проверим датасет

In [3]:
print("np_Small_BAD_test_FD001:")
print(np_Small_BAD_test_FD001)
print("\nnp_Small_GOOD_test_FD001:")
print(np_Small_GOOD_test_FD001)

np_Small_BAD_test_FD001:
[[     nan      nan      nan      nan]
 [ 1.0e+00  1.0e+00      nan  3.0e-04]
 [ 1.0e+00  2.0e+00 -2.7e-03 -3.0e-04]
 [ 1.0e+00  3.0e+00  3.0e-04      nan]
 [ 1.0e+00      nan  4.2e-03  0.0e+00]]

np_Small_GOOD_test_FD001:
[[     nan      nan      nan      nan]
 [ 1.0e+00  1.0e+00  2.3e-03  3.0e-04]
 [ 1.0e+00  2.0e+00 -2.7e-03 -3.0e-04]
 [ 1.0e+00  3.0e+00  3.0e-04  1.0e-04]
 [ 1.0e+00  4.0e+00  4.2e-03  0.0e+00]]


### Поиск пропусков в pd.Dataframe

In [ ]:
# def is_null(dataframe: pd.DataFrame):
#     count_nulls = 0
#     null_set = pd.Dadataframe(dataframe.isnull())

#     for col in dataframe.columns:
#         for val in dataframe[col].is_null():
#             if (val == True):
#                 count_nulls +=1
#             #print(val)
#         print("in ", col, "nulls: ", count_nulls)
#         count_nulls = 0

# is_null(np_Small_BAD_test_FD001)

### Создание и сохранение Pipeline

#### Создаем SimpleInputer - простой заполнитель пропущенных значений в датасете

Так как предполагается, что данные упорядоченны по циклам работы двигателя, не целесообразно использовать стандартный заполнитель SimpleImputer. разумнее будет вычислить среднее значение лишь до и после пропущенного значения в каждом столбце. Можно использовать KNNImputer.

In [11]:
simple_inputer = KNNImputer(n_neighbors = 2)
std_scaler = StandardScaler()

pipe_num = Pipeline([('imputer', simple_inputer),('scaler', std_scaler)])

#### Обучаем Pipeline

In [13]:
res_num = pipe_num.fit_transform(np_Small_BAD_test_FD001)
res_num

array([[ 0.        ,  0.        , -0.01370506,  0.15075567],
       [ 0.        , -1.58113883,  0.05482024,  1.6583124 ],
       [ 0.        ,  0.        , -1.52126178, -1.35680105],
       [ 0.        ,  1.58113883, -0.15075567, -0.60302269],
       [ 0.        ,  0.        ,  1.63090227,  0.15075567]])

Теперь можем применить полученный pipeline
Видим, что пропусков нет

In [ ]:
# res_num = pd.DataFrame(res_num, columns=pipe_num['scaler'].get_feature_names_out(np_Small_BAD_test_FD001[0,:]))
# res_num

res = pipe_num.transform(np_Small_BAD_test_FD001)

#### Сохраняем Pipline

In [18]:
savePath = 'C:/Users/Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/final/Pipelinr.pickle'

with open(savePath, 'wb') as handle:
    save_pik_pipeline = pickle.dumps(pipe_num)

In [18]:
inp_dir = 'C:/Users/Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/processed'
save_dir = f"{inp_dir}/"
files = os.listdir(inp_dir)

files

['new_1.txt']

In [4]:
for file in files:
    new_name = f'{save_dir}new_{file}'
    os.rename(save_dir + file, new_name)

#### Сохраняем pickle

In [ ]:
inp_dir = 'C:/Users/Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/raw/tests'
out_dir = 'C:/Users/Dmitriy/Desktop/Univer/Diplom/diplom_autoencoder/data/processed'

employ_Pipline(inp_dir,
               out_dir,
               pipe_num) 

### Преобразование data Frame

1. Добавляем столбец с индексами
2. Удаляем строку с названием столбцов
3. Удаляем строку, если в ней есть пропуск 

In [4]:
# 1 - Добавление столбца с индексами
def add_col_indexes(dataFrame: np.array) -> np.array:
    
    str_count, col_count = dataFrame.shape
    indexses = []
    indexses.append(0)

    print("str -->", str_count,
          "\ncol -->", col_count)
    
    for i in range(str_count-1):
        indexses.append(i)

    dataFrame = np.insert(dataFrame, 0, indexses, axis= 1)

    return dataFrame

In [5]:
# 2 - Удаление 0-й строки с названиями столбцов
def delete_col_names(dataFrame: np.array) -> np.array:
    dataFrame = dataFrame[1:, 0:]

    return dataFrame

In [41]:
def delete_nan_str(dataFrame: np.array) -> np.array:
    
    line, col  = dataFrame.shape
    res_dataFrame = np.empty((1, col))

    for line in dataFrame:

        line = np.array([line])
        print(line)

        if(check_nan_dataFrame(line) == False):
            res_dataFrame = np.vstack((res_dataFrame, line))
        else:
            continue

    res_dataFrame = delete_col_names(res_dataFrame)
    
    return res_dataFrame

delete_nan_str(np_Small_BAD_test_FD001)

[[nan nan nan nan]]
[[1.e+00 1.e+00    nan 3.e-04]]
[[ 1.0e+00  2.0e+00 -2.7e-03 -3.0e-04]]
[[1.e+00 3.e+00 3.e-04    nan]]
[[1.        nan 0.0042 0.    ]]


array([[ 1.0e+00,  2.0e+00, -2.7e-03, -3.0e-04]])

In [8]:
def to_standardization_df(dataFrame: np.array) -> np.array:
    
    dataFrame = add_col_indexes(dataFrame)
    dataFrame = delete_col_names(dataFrame)
    dataFrame = delete_nan_str(dataFrame)

    return dataFrame

to_standardization_df(np_Small_BAD_test_FD001)

str --> 5 
col --> 4


array([[ 0.0e+00,  1.0e+00,  1.0e+00,  2.3e-03,  3.0e-04],
       [ 1.0e+00,  1.0e+00,  2.0e+00, -2.7e-03, -3.0e-04],
       [ 2.0e+00,  1.0e+00,  3.0e+00,  3.0e-04,  1.0e-04],
       [ 3.0e+00,  1.0e+00,  4.0e+00,  4.2e-03,  0.0e+00]])

### Из NumPy в Pandas

In [ ]:
values = np_Small_BAD_test_FD001[1:,1:].astype(float)
colums = np_Small_BAD_test_FD001[0, 1:]

pd_Small_BAD_test_FD001 = pd.DataFrame(values, columns = colums)
pd_Small_BAD_test_FD001

### Ф-ии проверки данных / наличия nan

In [28]:
def check_nan_dataFrame(dataset: np.array) -> bool: #

   #dataset_v = np.delete(dataset, (0), axis=0)
    res: bool
    res = False

    for st in dataset:
        for col in st:
            # print(f"{col} --> {type(col)}")
            if(np.isnan(col)):
                res = True
                return res
    
    return res
            

check_nan_dataFrame(np_Small_GOOD_test_FD001)

True

### Функции для модуля

In [47]:
def employ_Pipline(     #df: np.array,
                        inp_dir: str,
                        out_dir: str,
                        pipline: Pipeline) -> bool:
        
        status_log = ["Preprocess data finished successfull", "Preprocess data finished error"]

        # Получаем все документы в папке
        inpFilesList = os.listdir(inp_dir)
        outFilesList = inpFilesList
        status = False
        
        # Проверяем датасет на пригодность
        # if check_type_np(df):
        #     print("Dataset is GOOD")
        # else:
        #     print("Dataset is BAD")
        #     status = False
        #     return status

        # for fl in inpFilesList:
        #     print(fl)

        # На конце выходной строки дирректории должна стоять "/"
        if inp_dir[-1] != '/':
            inp_dir = f"{inp_dir}/"
        if out_dir[-1] != '/':
            out_dir = f"{out_dir}/"

        # Применение
        try:
            for file in inpFilesList:
                if file == ".gitkeep":
                    continue
                    
                print("    Processed -> ", file)

                dataframe = genfromtxt(inp_dir, delimiter=',')

                # Добавляем столбец индексов и удаляем 0-ю строку с названиями столбцов
                str_count, col_count = file.shape
                indexses = []
                indexses.append(0)
                
                for i in range(str_count-1):
                    indexses.append(i)
                    print(indexses)

                np_Small_BAD_test_FD001 = np.insert(np_Small_BAD_test_FD001, 0, indexses, axis= 1)
                np_Small_BAD_test_FD001 = np_Small_BAD_test_FD001[1:, 0:]

                dataFrame = (inp_dir + file)
                newDataFrame = pipline.fit_transform(dataFrame)
                #newDataFrame = pd.DataFrame(dataFrame, columns=pipline['scaler'].get_feature_names_out(dataFrame.columns))
                
                # Сохранение
                filename, extension = os.path.splitext(file)
                #newDataFrame.to_pickle(f"{out_dir}new_{filename}.pickle")
                newDataFrame.save(f"{out_dir}new_{filename}", arr = newDataFrame, allow_pickle = True)
            
            print(status_log[0])
            status = True
            return status
        
        except:
            print(status_log[1])
            status = False
            return status
    


employ_Pipline(#df =np_Small_GOOD_test_FD001,
                inp_dir = General_path + raw + tests,
                out_dir = General_path + processed,
                pipline = pipe_num)

Dataset is BAD


False

### Источники
1 - https://habr.com/ru/companies/yandex_praktikum/articles/756474/